<a href="https://colab.research.google.com/github/reidnersousa/Estudo/blob/cuda/SomaVetorialEx3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile somaVetorial.cu

#include<stdio.h>
#include<time.h>
#include<stdlib.h>

__global__ void FazTudoInt (int *v1, int *v2 ,int *v3 , int *v4 )
{

	
  v2[blockIdx.x]=v2[blockIdx.x]+ v3[blockIdx.x];
  v4[blockIdx.x]=v2[blockIdx.x]* v4[blockIdx.x];
  v3[blockIdx.x]=v4[blockIdx.x]/ v3[blockIdx.x];
  v1[blockIdx.x]=v3[blockIdx.x]-50;
}


__global__ void add(int *a, int *b, int *soma) {
	soma[blockIdx.x] = a[blockIdx.x] + b[blockIdx.x];
}

__global__ void sub(int*a,int*b,int*sub){
  sub[blockIdx.x] = a[blockIdx.x] - b[blockIdx.x];
}



void random_ints(int *a, int N){
	int i;

	for (i=0;i < N; i++){
		a[i] = rand() % 100;
    printf("%3d ",a[i]);
	}
  puts("");
}



#define N 1000

int main(void) {
	int  *b, *c,*d , *soma ; // host copies of  b, c,d,soma
	int *d_a, *d_b, *d_c,*d_d; // device copies of a, b, c
	int *soma_d ; 
  int size = N * sizeof(int);
	int i;


	// Alloc space for device copies of a, b, c
	cudaMalloc((void **)&d_a, size);
	cudaMalloc((void **)&d_b, size);
	cudaMalloc((void **)&d_c, size);
  cudaMalloc((void **)&d_d, size);

  cudaMalloc((void **)&soma_d, size);


	srand(time(NULL));
	
	// Alloc space for host copies of a, b, c and setup input values
	b = (int *)malloc(size); random_ints(b, N);
  c = (int*)malloc(size) ; random_ints(c,N);
  d = (int*)malloc(size) ; random_ints(d,N);

	
  soma = (int *)malloc(size);
	
	
  
	// Copy inputs to device
	
	cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_d, d, size, cudaMemcpyHostToDevice);
	
	// Launch add() kernel on GPU with N blocks
	FazTudoInt<<<N,1>>>(d_a, d_b, d_c,d_d);
	
	// Copy result back to host
	cudaMemcpy(soma, d_a, size, cudaMemcpyDeviceToHost);
	
  

	for (i = 0; i< N-1; i++){
		printf(">>%d,",soma[i]); //'soma' e equivalente a 'a' no execicio 
	}
	printf("==%d\n",soma[i]);


	// Cleanup
	 free(b); free(c);free(d);
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c); cudaFree(d_d);
	
	
	return 0;
	}

Writing somaVetorial.cu


In [2]:
!nvcc somaVetorial.cu -o somaVetorial
!./somaVetorial

  0  72  25  16  25  18  71   6  80  60  26   9  25  95  28  97   6   9  63   0  63  53  88  75  57   8  59  32  58  15  21  58  87  98  27  64  68  98  70   1  59  48  62  84  96  91  33   2  52  48   3  15   1  43  90  11  51   2  95  61  17  16  19   5  66  98  69  34  97  92  35   8  40  98  44  36  41  77  91  93  25  94  60  78  37   3  41  40   5  36   1  22   4  72  79  70  71  49  57  68  93  92  28  33  42  72  22  83  49  13  76  26  59  89  56  96  92  98  36  97  34  37  71  91   9  51  61  32  52  70  52  45  63  80  30  57  52   4  41  53  69  69  79  28  58  88  24   2  86  12  51  72  49  23  63  11  26  77  43  78  47  96  75  62  28  57  20  81  14  61  34  83  82  66  12  41  54  88  95  92   1  47  64   2  70  80  65  48  57  61  78  56   9  53  71  37  62  91  18  76   4   5  12  86  23  76  79  29  64  75  21  17  22  37  72  44  17  37  44  26  98  22  35   7  27   6  97  89  49  15  18  53  72  30  39  95  58  71  76  22  46  97  92  20  87  64  16  56   1  60 